<h1 style="text-align: center;">Syntactic Analysis - Tutorial</h1> 

**CFGs** are Simple rules that can build, analyze, and understand sentence structures.
**CFGs** are used for:
* Syntax checkers (compilers: C++, Python)
* Code parsers to analyse the syntactic structure of source code.
* Chatbots and Grammar-based NLP
* Auto-correct and Grammar checkers

## Parsing with CFG

In [1]:
import nltk
from nltk import CFG
import string

# Expanded grammar
grammar = CFG.fromstring("""
  S -> NP VP
  NP -> Pronoun | Det Noun | NP PP
  VP -> Verb NP | VP PP
  PP -> Prep NP

  Pronoun -> 'i'
  Det -> 'the' | 'a'
  Noun -> 'cat' | 'dog' | 'man' | 'telescope'
  Verb -> 'chased' | 'found' | 'had' | 'saw'
  Prep -> 'with'
""")

parser = nltk.ChartParser(grammar)

def preprocess(sentence):
    sentence = sentence.lower()
    sentence = ''.join(c for c in sentence if c not in string.punctuation) # Remove punctuation
    return sentence.split() # Split into tokens
    

sentence = "The dog chased the cat."
# sentence = "I saw the man with the telescope."

tokens = preprocess(sentence)

# Parse and print all possible trees
for tree in parser.parse(tokens):
    tree.pretty_print()
    


               S                
      _________|_____            
     |               VP         
     |          _____|___        
     NP        |         NP     
  ___|___      |      ___|___    
Det     Noun  Verb  Det     Noun
 |       |     |     |       |   
the     dog  chased the     cat 



## Multiple nestings of prepositions

* An example of a noun phrase NP sentence with multiple preposition nestings.
* What happens if we use S -> VP NP only?

In [2]:
import nltk
from nltk import CFG

# Define a recursive grammar
grammar = CFG.fromstring("""
  S -> VP NP | NP
  NP -> Det Noun | NP PP
  VP -> Verb NP
  PP -> Prep NP

  Det -> 'the'
  Noun -> 'key' | 'shelf' | 'door' | 'lock'
  Verb -> 'found'
  Prep -> 'on' | 'by' | 'with'
""")

# Create a parser
parser = nltk.ChartParser(grammar)

# Sentence with multiple preposition nestings
sentence = "the key on the shelf by the door with the lock".split() # No Verb!

# Parse and show only the first tree
trees = list(parser.parse(sentence))

if trees:
    tree = trees[0]
    tree.pretty_print()
else:
    print("No parse tree found.")


                                 S                                 
                                 |                                  
                                 NP                                
                    _____________|______________________            
                   NP                                   |          
               ____|__________________                  |           
              NP                      |                 |          
      ________|____                   |                 |           
     |             PP                 PP                PP         
     |         ____|___           ____|___          ____|___        
     NP       |        NP        |        NP       |        NP     
  ___|___     |     ___|____     |     ___|___     |     ___|___    
Det     Noun Prep Det      Noun Prep Det     Noun Prep Det     Noun
 |       |    |    |        |    |    |       |    |    |       |   
the     key   on  the     shelf  by  the 

## CFG Mini Grammar for Commands

The assistant (e.g., **smart home assistant**) needs to parse the command to know:

* Action (Verb): what to do (open, close, start)
* Target (Noun): which device or thing to control (door, window, app)

In [3]:
import nltk
from nltk import CFG

# Define the correct grammar
command_grammar = CFG.fromstring("""
  S -> VP
  VP -> Verb NP
  Verb -> 'open' | 'close' | 'start' | 'turn_on' | 'turn_off'
  NP -> Det Noun
  Det -> 'the'
  Noun -> 'door' | 'window' | 'app' | 'security_lock'
""")

# Create a parser
parser = nltk.ChartParser(command_grammar)

# Input command
commands = 'open the door'.split()
# commands = "turn_on the security_lock".split()

# Parse and print trees
for tree in parser.parse(commands):
    tree.pretty_print()
    

      S          
      |           
      VP         
  ____|___        
 |        NP     
 |     ___|___    
Verb Det     Noun
 |    |       |   
open the     door



In [4]:
verb = None
noun = None

for subtree in tree.subtrees():
    if subtree.label() == 'Verb':
        verb = ' '.join(subtree.leaves())
    if subtree.label() == 'Noun':
        noun = ' '.join(subtree.leaves())

print(f"Action: {verb}, Target: {noun}")

Action: open, Target: door
